In [73]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.mixture import GaussianMixture
from keras.models import Sequential
from keras.layers import Dense,BatchNormalization
from keras import optimizers,regularizers,initializers

In [58]:
def import_dataset(normalised=True,scaleMethod='Standard'):
    '''
    Imports Dataset and returns either scaled values depending upon user inputs
    
    Input:
        normalised -- boolean depending upon whether the user wants to scale the values
        scaleMethod -- Type of scaler to be used if normalised is True
    
    Output:
        (X_train,X_test,Y_train,Y_test) -- the training and testing dataset
        scaler -- used to perform inverse transform if dataset is scaled
    '''
    data = pd.read_csv('MLDataset.csv')
    dataS = data.drop('Unnamed: 0',axis=1)
    
    if normalised == False:
        scaler = 'None'
        X = dataS.iloc[:,0:3].values
        Y = dataS.iloc[:,3:].values
        X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15,random_state=0)
        
    elif scaleMethod == 'Standard':
        scaler = StandardScaler()
        scaler.fit(dataS)
        dataS = scaler.transform(dataS)
        X = dataS[:,0:3]
        Y = dataS[:,3:]
        X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15,random_state=0)
    
    elif scaleMethod == 'MinMax':
        scaler = MinMaxScaler(feature_range=(-1,1))
        scaler.fit(dataS)
        dataS = scaler.transform(dataS)
        X = dataS[:,0:3]
        Y = dataS[:,3:]
        X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15,random_state=0)
    
    return X_train,X_test,Y_train,Y_test,scaler

In [3]:
def inverseTransform(scaler,*arr):
    '''
    Used to perform Inverse Transformation on normalised dataset
    
    Input:
        scaler -- Instance of Normaliser used
        *arr -- list of arrays to be concatenated
    '''
    data = np.concatenate(arr,axis=1)
    data = pd.DataFrame(data)
    arrInverse = scaler.inverse_transform(data)
    
    return arrInverse

In [4]:
def cost(y_test,y_pred):
    '''
    Calculates error of the model
    '''
    error = (y_test-y_pred)/y_test
    error = np.sum(abs(error))/(y_test.shape[0]*y_test.shape[1])*100
    
    return error

## K Neighbors Regressor 

In [5]:
Rscore = {}
for normalise in [True,False]:
    X_train,X_test,Y_train,Y_test,scaler = import_dataset(normalised=normalise)
    for nbr in range(2,100):
        for wgt in ['uniform', 'distance']:
            Model = KNeighborsRegressor(n_neighbors=nbr,weights=wgt)
            Model.fit(X_train,Y_train)
            Ytr_pred = Model.predict(X_train)
            Yts_pred = Model.predict(X_test)
            
            if normalise == False:
                error = cost(Y_test,Yts_pred)
                error_tr = cost(Y_train,Ytr_pred)
            else:
                true = inverseTransform(scaler,X_train,Y_train)
                pred = inverseTransform(scaler,X_train,Ytr_pred)
                
                error_tr = cost(true[:,3:],pred[:,3:])
                
                true = inverseTransform(scaler,X_test,Y_test)
                pred = inverseTransform(scaler,X_test,Yts_pred)
                
                error = cost(true[:,3:],pred[:,3:])
#             valTst = Model.score(X_test,Y_test)
#             val = Model.score(X_train,Y_train)
            param = {normalise,nbr,wgt}
            Rscore[(error,error_tr)]=param

In [6]:
Rscore

{(3.6773701923764315, 2.0377406344662994): {2, True, 'uniform'},
 (3.679338992146946, 0.019657641549745884): {2, True, 'distance'},
 (3.581132969907948, 2.4680730643993907): {3, True, 'uniform'},
 (3.5856075691560143, 0.018945020914940647): {3, True, 'distance'},
 (3.5308470372196337, 2.6799119232411908): {4, True, 'uniform'},
 (3.5303542706848052, 0.018791570734600898): {4, True, 'distance'},
 (3.5121478620912305, 2.818338409589552): {5, True, 'uniform'},
 (3.5095533210724277, 0.018728248976286048): {5, True, 'distance'},
 (3.4879022319495108, 2.908322518322019): {6, True, 'uniform'},
 (3.4902117049212666, 0.018676880339855085): {6, True, 'distance'},
 (3.4928574083991255, 2.975962807342564): {7, True, 'uniform'},
 (3.4898006920101046, 0.018667271959876615): {7, True, 'distance'},
 (3.496527800604853, 3.0310092507248343): {8, True, 'uniform'},
 (3.489089246191342, 0.018667271959876615): {8, True, 'distance'},
 (3.479766794201068, 3.0717959037923466): {9, True, 'uniform'},
 (3.47518896

## Gaussian Mixture Model

In [7]:
Rscore = {}
for normalise in [True,False]:
    X_train,X_test,Y_train,Y_test,scaler = import_dataset(normalised=normalise)
    for cpt in range(2,4):
        for cvr in ['full', 'tied','diag','spherical']:
            Model = GaussianMixture(n_components=cpt,covariance_type=cvr)
            Model.fit(X_train,Y_train[:,[0]])
            Ytr_pred = Model.predict(X_train)
            Yts_pred = Model.predict(X_test)
            Ytr_pred = Ytr_pred.reshape(Ytr_pred.shape[0],1)
            Yts_pred = Yts_pred.reshape(Yts_pred.shape[0],1)
            if normalise == False:
                error = cost(Y_test[:,[0]],Yts_pred[:,[0]])
                error_tr = cost(Y_train[:,[0]],Ytr_pred[:,[0]])
            else:
                true = inverseTransform(scaler,X_train,Y_train)
                pred = inverseTransform(scaler,X_train,Ytr_pred,Y_train[:,1:])
                
                error_tr = cost(true[:,3:],pred[:,3:])
                
                true = inverseTransform(scaler,X_test,Y_test)
                pred = inverseTransform(scaler,X_test,Yts_pred,Y_test[:,1:])
                
                error = cost(true[:,3:],pred[:,3:])
#             valTst = Model.score(X_test,Y_test)
#             val = Model.score(X_train,Y_train)
            param = {normalise,cpt,cvr}
            Rscore[(error,error_tr)]=param

In [9]:
Rscore

{(1.2660092323866023, 1.24709715142702): {2, True, 'full'},
 (1.3795775748341284, 1.3736157175060968): {2, True, 'tied'},
 (1.1258963890878102, 1.1369794626085483): {2, True, 'diag'},
 (1.3550035319133946, 1.3487678495161903): {2, True, 'spherical'},
 (1.6296353958208654, 1.6553384724433031): {3, True, 'full'},
 (1.3877418659182346, 1.4230583718113954): {3, True, 'tied'},
 (1.6117110666233059, 1.6281736313921367): {3, True, 'diag'},
 (1.7631838510860192, 1.7588711679611089): {3, True, 'spherical'},
 (99.84412447428427, 99.84298182756302): {2, False, 'full'},
 (99.93263898075874, 99.92987012490183): {2, False, 'tied'},
 (99.95150540934567, 99.95150851608659): {2, False, 'diag'},
 (99.86495075170222, 99.86282165749554): {2, False, 'spherical'},
 (99.74949729922261, 99.75188220846508): {3, False, 'full'},
 (99.81223158195307, 99.80756789724285): {3, False, 'tied'},
 (99.64331331754973, 99.64693625918024): {3, False, 'diag'},
 (99.71357609508574, 99.71552151080657): {3, False, 'spherical'}

## Artificial Neural Network

In [59]:
X_train,X_test,Y_train,Y_test,scaler = import_dataset(scaleMethod='MinMax')

In [75]:
model = Sequential()
model.add(Dense(10, input_dim=X_train.shape[1],
                kernel_initializer=initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None), 
                activation='relu',
                kernel_regularizer=regularizers.l2(1e-4),
                bias_regularizer=regularizers.l2(1e-4),
                activity_regularizer=regularizers.l2(1e-5)))
model.add(BatchNormalization())
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(Y_train.shape[1], activation='tanh'))
opt = optimizers.Adam(learning_rate=0.01)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=100, batch_size=100)
met = model.evaluate(X_test, Y_test)

Epoch 1/100
170/170 [==============================] - 0s 2ms/step - loss: 0.2837 - accuracy: 0.2624
Epoch 2/100
170/170 [==============================] - 0s 2ms/step - loss: 0.2591 - accuracy: 0.2565
Epoch 3/100
170/170 [==============================] - 0s 2ms/step - loss: 0.2541 - accuracy: 0.2603
Epoch 4/100
170/170 [==============================] - 0s 2ms/step - loss: 0.2531 - accuracy: 0.2669
Epoch 5/100
170/170 [==============================] - 0s 2ms/step - loss: 0.2532 - accuracy: 0.2678
Epoch 6/100
170/170 [==============================] - 0s 2ms/step - loss: 0.2505 - accuracy: 0.2772
Epoch 7/100
170/170 [==============================] - 0s 2ms/step - loss: 0.2540 - accuracy: 0.2745
Epoch 8/100
170/170 [==============================] - 0s 2ms/step - loss: 0.2515 - accuracy: 0.2781
Epoch 9/100
170/170 [==============================] - 0s 2ms/step - loss: 0.2490 - accuracy: 0.2782
Epoch 10/100
170/170 [==============================] - 0s 2ms/step - loss: 0.2516 - accura

KeyboardInterrupt: 